In [1]:
import requests
from scrapy import Selector
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

In [2]:
url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1'
driver = webdriver.Chrome('./chromedriver.exe')
driver.get(url)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16560\1707358220.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [3]:
Products = {'URL':[],'Name':[],'Price':[],'Rating':[],'Reviews':[],'Description':[],'ASIN':[],'Manufecturer':[]}
for i in tqdm(range(1,20)):
    sel = Selector(text=driver.page_source)
    links = ['https://www.amazon.in/'+i.xpath('.//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]/@href').extract_first() for i in sel.xpath('//div[@data-component-type="s-search-result"]') if i.xpath('.//span[@class="a-color-base"]/text()').extract_first()!='Sponsored']
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    for l in links:
        driver.get(l)
        time.sleep(1)
        sel2 = Selector(text=driver.page_source)
        Products['URL'].append(l) # Product Url
        Products['Name'].append(sel2.xpath('//span[@id="productTitle"]/text()').extract_first()) # Product Name
        Products['Price'].append(sel2.xpath('//span[@class="a-price-whole"]/text()').extract_first()) # Product Price
        Products['Rating'].append(sel2.xpath('//span[@class="a-icon-alt"]/text()').extract_first()) # Rating
        Products['Reviews'].append(sel2.xpath('//span[@id="acrCustomerReviewText"]/text()').extract_first()) # Number of Reviews
        Products['Description'].append(''.join(sel2.xpath('.//ul[@class="a-unordered-list a-vertical a-spacing-mini"]//text()').extract()))
        Products['ASIN'].append(sel2.xpath('//th[contains(text(),"ASIN")]/following-sibling::td/text()').extract_first())
        Products['Manufecturer'].append(sel2.xpath('//div[@id="merchant-info"]/a/span/text()').extract_first())
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.find_element('xpath','//a[contains(text(),"Next")]').click()
    time.sleep(2)

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [39:43<00:00, 125.47s/it]


In [4]:
df = pd.DataFrame(Products)

In [5]:
df['Reviews'] = df['Reviews'].str.replace('(','').str.replace(')','')

C:\Users\DELL\AppData\Local\Temp\ipykernel_16560\1145593625.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Reviews'] = df['Reviews'].str.replace('(','').str.replace(')','')


In [6]:
df.to_csv('Products.csv',index=False)